### Word Group Video Select

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import glob
from gtts import gTTS
from PIL import Image, ImageFilter, ImageOps
#from playsound import playsound
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips,CompositeVideoClip, CompositeAudioClip
import pandas as pd
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import cv2

#### Adjust Image File Convert Video And Concat 

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

filename_length_num = 5

# lang
if lang_folder == "English":
    lang = 'en'
elif lang_folder == "Turkish":
    lang = 'tr'
elif lang_folder == "German":
    lang = 'de'
elif lang_folder == "French":
    lang = 'fr'
elif lang_folder == "Spanish":
    lang = 'es'
elif lang_folder == "Arabic":
    lang = 'ar'
elif lang_folder == "Portuguese":
    lang = 'pt'
elif lang_folder == "Dutch":
    lang = 'nl'
elif lang_folder == "Italian":
    lang = 'it'
else:
    pass

print(f"{lang_folder}\n{lang}") 

Turkish
tr


In [3]:
# Data path and folder


##### Read Data

In [5]:
df_deployment_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/Manuel Select/\
Revised Ngrams All in One.xlsx", sheet_name="Sheet2")
df_deployment_file.drop_duplicates(inplace=True)
df_deployment_file.reset_index(drop=True, inplace=True)
df_deployment_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,yardım eder misin,eder misin
2,bana yardım eder misin,yardım eder misin,yardım eder
3,başka bir şey daha,başka bir şey,bir şey
4,başka bir şey daha,başka bir şey,başka bir
...,...,...,...
475,NaN,çok kötü bir,çok kötü
476,NaN,bir şey olmaz,NaN
477,NaN,şuna bir bak,bir bak
478,NaN,şuna bir bak,NaN


In [ ]:
# image folder
image_folder = "Genome_Picture_In_Picture_Rename_Select3"
selected_column = "fourgram"

In [ ]:
# Output path and folder
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3"  # Fourgram_Selected, Test2
result_folder = f"{selected_column.capitalize()}3"  # Twogram, Threegram, Fourgram, Sample


In [ ]:
# ///***
output_path = f"{result_path}/{main_result_folder}/{result_folder}"  # output path
text_audio_path = f"{result_path}/{main_result_folder}/{result_folder}/Data/Text_Audio"

Path(output_path).mkdir(parents=True, exist_ok=True)
Path(text_audio_path).mkdir(parents=True, exist_ok=True)
# ****\\\

In [ ]:
# twogram threegram fourgram selection will add

In [ ]:
word_group_all = df_deployment_file[f"{selected_column}"].dropna().to_list()
word_group_list = list(set(word_group_all))
len(word_group_list)

#### Result Video File Concat

In [ ]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import nltk
from nltk import word_tokenize

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

filename_length_num = 5

In [ ]:
def concat_video2(text_list, video_data_folder_name, video_data_path, out_folder_name, out_path, filename):  
    '''
    concat_video(text_list, video_data_folder_name, video_data_path, out_folder_name, out_path, filename)\n
    text_list is list values. video_data_folder_name, video_data_path, out_folder_name and out_path are path and folder in string(str)\n
    filename is string of result mp4 file name.
    ex.\n
    path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/200 Words"\n
    audio_image_video_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Data/200 Words"\n
    concat_video(word_select_list, "Image_Video_Audio", audio_image_video_path, "Concat_Video", path, "200_Word_Combined_Sample")
    '''
    no_image_file_list = []
    result_path = f"{out_path}/{out_folder_name}"
    Path(result_path).mkdir(parents=True, exist_ok=True)
    video_text_list = []
    video_list = []

    if isfile(f"{result_path}/{filename}.mp4"):
        pass
    else:
        for text in text_list:
            if isfile(f"{video_data_path}/{video_data_folder_name}/{text}.mp4"):
                video_text_list.append(text)            
            else:
                no_image_file_list.append(text)

        for video_num in range(len(video_text_list)):
            try:
                clip_var = VideoFileClip(f"{video_data_path}/{video_data_folder_name}/{video_text_list[video_num]}.mp4", target_resolution=(900, 1600))  # height, width
                video_list.append(clip_var)            
            except:
                print("Oops!", sys.exc_info()[0], "occurred.", f"{video_text_list[video_num]}.mp4")

        combined = concatenate_videoclips(video_list, method='compose')
        combined.write_videofile(f"{result_path}/{filename}.mp4", audio_bitrate='128k', audio_fps=44100, audio_codec="aac")
    
    return no_image_file_list

In [ ]:
# Previous output path and folder
#prev_result_path = f"/home/kurubal/Documents/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/200 Words"
#prev_main_result_folder = "Twogram_Threegram"
#prev_result_folder = "Threegram_Twogram_Selected"

prev_result_path = f"{result_path}"
prev_main_result_folder = f"{main_result_folder}"
prev_result_folder = f"{result_folder}"

In [ ]:
data_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/Result"  # previous output path result for data for this
result_path = f"{prev_result_path}/{prev_main_result_folder}/{prev_result_folder}/Result"

In [ ]:
#df_deployment_file = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/\
#Deployment2/Result/Deploy2 Result Manuel/Turkish/Twogram_Threegram_Selected.xlsx", sheet_name="twogram_threegram_selected")
#df_deployment_file

In [ ]:
#word_group_list = df_deployment_file["twogram_threegram_selected"].dropna().to_list()
#word_group_list

In [ ]:
len(word_group_list)

In [ ]:
word_group_list2 = []
for word_group in word_group_list:
    word_list = word_tokenize(word_group)
    if len(word_list)>filename_length_num:
        filename_new = " ".join(word_list[0:filename_length_num])
    else:
        filename_new = word_group
    word_group_list2.append(filename_new)

In [ ]:
concat_video2(word_group_list2, "Concat_Video", data_path, "All_Concat_Video", result_path, f"{prev_result_folder}")